## Parte A - Consultas de Spark

En este notebook contestaremos a las preguntas presentadas en la Parte A utilizando consultas de Spark. 

Para esta parte utilizaremos los datos que hemos guardado en el notebook de *Limipieza* en formato parquet. 

In [ ]:
%%info

In [ ]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lower, trim, regexp_replace
from pyspark.sql.window import Window

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1746552940098_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Abrimos sesión de Spark
spark = SparkSession.builder \
    .appName("profeco_Parte_A") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# bucket
#NAME = 'thmrudolf' ##CAMBIAR AQUÍ SU NOMBRE.
NAME = 'marta'
BUCKET = f"s3://itam-analytics-{NAME}"
FOLDER = 'profeco'


# type of catalog
CATALOG_TYPE = 'basicos'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.functions import asc, desc

#Leemos los archivos de la carpeta parquet
df_parquet = spark.read.parquet(f"{BUCKET}/{FOLDER}/parquet")
df_parquet.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- producto: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- precio: float (nullable = true)
 |-- fecha: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- anio: integer (nullable = true)

## Preguntas Generales

Comenzamos constestando las preguntas generales, referidas a todos los catálogos y todos los estados. 

Queremos un análisis **(por año)** y sobre todos los catálogos.

* ¿Cuántos catálogos diferentes tenemos?
* ¿Cuáles son los 20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.
* ¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?
* ¿Cuántas observaciones tenemos por estado?
* De cada estado obten: el número de catalogos diferentes por año, ¿ha  aumentado el número de catálogos con el tiempo?


### Catálogos diferentes:
Contestaremos a las preguntas referentes a número de catálogos y catálogos con más observaciones. 

Para número de catálogos, veremos el total, luego número de catálogos por año, y finalmente qué catálogos se registraron en cada año. 

In [ ]:
#¿Cuántos catálogos diferentes hay?
total_catalogos = df_parquet.select("catalogo").distinct().count()
print(f"El total de catalogos distintos es: {total_catalogos}")
#Veamos cuáles son sus nombres
catalogos_name = df_parquet.select("catalogo").distinct().show()
#Nº de catálogos por año:
df_catalogo_anio = df_parquet.groupBy("anio")\
    .agg(F.countDistinct("catalogo").alias("num_catalogos"))\
    .orderBy("anio")\
    .show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

El total de catalogos distintos es: 12
+-------------------+
|           catalogo|
+-------------------+
|            basicos|
|        aeropuertos|
|       medicamentos|
|  electrodomesticos|
|   utiles escolares|
|           juguetes|
|           mercados|
| frutas y legumbres|
|              pacic|
|          navidenos|
|pescados y mariscos|
|              tenis|
+-------------------+

+----+-------------+
|anio|num_catalogos|
+----+-------------+
|2018|           10|
|2019|           11|
|2020|           10|
|2021|           10|
|2022|           11|
|2023|           11|
|2024|           10|
+----+-------------+

A pesar de que el total de catálogos es 12, vemos que en ningún año se registran los 12 catálogos. Es decir en cada año, siempre faltan uno o dos catálgos sin registrar.  Vamos a ver cuáles son

In [ ]:
#Cuales son esos catálogos
df_parquet.groupBy("anio")\
    .agg(F.collect_set("catalogo").alias("catalogos"))\
    .orderBy("anio")\
    .show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------------------------------------------------------------------------------------------------------------------------------------------+
|anio|catalogos                                                                                                                                               |
+----+--------------------------------------------------------------------------------------------------------------------------------------------------------+
|2018|[pescados y mariscos, juguetes, utiles escolares, frutas y legumbres, navidenos, electrodomesticos, medicamentos, tenis, basicos, mercados]             |
|2019|[pescados y mariscos, juguetes, utiles escolares, aeropuertos, frutas y legumbres, navidenos, electrodomesticos, medicamentos, tenis, basicos, mercados]|
|2020|[pescados y mariscos, juguetes, utiles escolares, aeropuertos, frutas y legumbres, navidenos, electrodomesticos, medicamentos, basicos, mercados]       |
|2021|[pescados y mariscos, juguetes, ut

### Catálogos con más observaciones

Vemos también de forma general, cuántas observaciones por catálogo en todos los años observados y también disgregados por año. De esta forma, observamos más rápidamente en qué año se tuvo registro de qué catálogo y cuántos registros se hicieron. 

In [ ]:
#Catálogos con más observaciones
#Totales
df_parquet.groupBy("catalogo") \
    .agg(F.count("*").alias("num_observaciones")) \
    .orderBy(desc("num_observaciones")) \
    .show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------------+
|catalogo           |num_observaciones|
+-------------------+-----------------+
|basicos            |46965358         |
|medicamentos       |19207468         |
|electrodomesticos  |7175494          |
|frutas y legumbres |5041527          |
|utiles escolares   |2936010          |
|mercados           |2238608          |
|juguetes           |1432183          |
|pacic              |1079162          |
|pescados y mariscos|569519           |
|navidenos          |236543           |
|tenis              |15768            |
|aeropuertos        |581              |
+-------------------+-----------------+

In [ ]:
#Observaciones por año. 
df_parquet.groupBy("catalogo", "anio") \
    .agg(F.count("*").alias("num_observaciones")) \
    .orderBy(desc("num_observaciones")) \
    .show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----+-----------------+
|catalogo          |anio|num_observaciones|
+------------------+----+-----------------+
|basicos           |2018|8802747          |
|basicos           |2022|7945081          |
|basicos           |2023|7742108          |
|basicos           |2019|7398860          |
|basicos           |2021|6919470          |
|basicos           |2020|6170515          |
|medicamentos      |2018|3657621          |
|medicamentos      |2023|3192617          |
|medicamentos      |2022|3175406          |
|medicamentos      |2021|2932272          |
|medicamentos      |2019|2883566          |
|medicamentos      |2020|2567358          |
|basicos           |2024|1986577          |
|electrodomesticos |2018|1735292          |
|electrodomesticos |2019|1352068          |
|electrodomesticos |2023|1183304          |
|electrodomesticos |2022|1029302          |
|electrodomesticos |2021|1004833          |
|frutas y legumbres|2018|938411           |
|frutas y legumbres|2022|926031 

 ### Top 20 observaciones: 
 Para cada catálogo, obtenemos la lista de los productos con más observaciones. 
 
 Esto lo hacemos de forma global y desagregada por año. 

In [ ]:
#Top 20 productos por catálogo de forma global
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#Primero hacemos la búsqueda, y la guardamos en un dataframe. 

# Paso 1: Contamos observaciones por producto y catálogo
df_producto_catalogo = df_parquet.groupBy("catalogo", "producto") \
    .agg(F.count("*").alias("num_observaciones"))

# Paso 2: Definimos ventana para rankear productos por catálogo
window_catalogo = Window.partitionBy("catalogo").orderBy(F.desc("num_observaciones"))

# Paso 3: Asignamos ranking y filtramos top 20
df_top20 = df_producto_catalogo.withColumn("rank", F.row_number().over(window_catalogo)) \
                               .filter(F.col("rank") <= 20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Imprimimos el resultado, de forma legible

catalogos = df_top20.select("catalogo").distinct().orderBy("catalogo").rdd.flatMap(lambda x: x).collect()

for catalogo in catalogos:
    print(f"\n\n{'='*50}")
    print(f"CATÁLOGO: {catalogo.upper()}")
    print(f"{'='*50}")
    
    df_top20.filter(F.col("catalogo") == catalogo) \
            .orderBy("rank") \
            .select("producto", "num_observaciones") \
            .show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



CAT?LOGO: AEROPUERTOS
+-----------------------------------+-----------------+
|producto                           |num_observaciones|
+-----------------------------------+-----------------+
|refresco                           |114              |
|pastelillos y pan dulce empaquetado|99               |
|desodorante                        |76               |
|agua con gas                       |30               |
|jabon de tocador                   |29               |
|leche ultrapasteurizada            |29               |
|cacahuates                         |23               |
|talco                              |18               |
|galletas dulces                    |18               |
|agua sin gas                       |17               |
|sterimar                           |17               |
|salsa picante                      |16               |
|galletas saladas                   |16               |
|picot. sal de uvas                 |16               |
|lamisil                

#### Top 20 observaciones por año y catálogo

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#CONSULTA
# Agrupamos por año, catálogo y producto, y contamos observaciones
df_productos = df_parquet.groupBy("anio", "catalogo", "producto") \
    .agg(F.count("*").alias("num_observaciones"))

# Definimos ventana por año y catálogo, ordenando por observaciones
window_anio_catalogo = Window.partitionBy("anio", "catalogo").orderBy(F.desc("num_observaciones"))

# Asignamos ranking dentro de cada (anio, catálogo)
df_top20 = df_productos.withColumn("rank", F.row_number().over(window_anio_catalogo)) \
                       .filter(F.col("rank") <= 20) \
                       .orderBy("anio", "catalogo", "rank")
            

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#IMPRESIÓN
# Colectamos los resultados al driver para impresión legible
rows = df_top20.select("anio", "catalogo", "producto", "num_observaciones", "rank").collect()

from itertools import groupby

# Agrupamos primero por año, luego por catálogo
for anio, grupo_anio in groupby(rows, key=lambda r: r['anio']):
    print(f"\n\n{'='*50}")
    print(f"AÑO: {anio}")
    print(f"{'='*50}")

    for catalogo, grupo_catalogo in groupby(grupo_anio, key=lambda r: r['catalogo']):
        print(f"\n\n{'='*50}")
        print(f"CATÁLOGO: {catalogo.upper()}")
        print(f"{'='*50}")
        print(f"{'Producto':<40} {'Observaciones':>15}")
        print("-" * 55)
        for row in grupo_catalogo:
            print(f"{row['producto']:<40} {row['num_observaciones']:>15}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



A?O: 2018


CAT?LOGO: BASICOS
Producto                                   Observaciones
-------------------------------------------------------
refresco                                          354589
shampoo                                           237228
jamon                                             233888
detergente p/ropa                                 209852
jabon de tocador                                  189343
desodorante                                       183596
leche ultrapasteurizada                           175356
salchicha                                         167120
chiles                                            155769
yoghurt                                           152462
cerveza                                           142285
leche en polvo                                    140941
toalla femenina                                   139586
mayonesa                                          134973
carne res                                         133980


In [ ]:
#Guardamos el resultado en un bucket

df_top20.write.mode("overwrite").parquet(f"{BUCKET}/{FOLDER}/catalogos_top_20")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Datos por estado. 

Primero vamos a ver que tenemos información de todos los estados de la República:

In [ ]:
#¿Tenemos datos de todos los estados del país?
no_estados = df_parquet.select("estado").distinct().count()
if (no_estados < 32):
    print("No hay información de todos los estados")
elif (no_estados > 32):  
    print("PROBLEMA: Hay más de 32 estados\n Revisa la limpieza de datos")
else: 
    print("Hay información de todos los estados")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Hay informaci?n de todos los estados

Imprimimos ahora los nombres de los estados para revisar que todo lo está haciendo correcto. 

In [ ]:
#Mostramos los estados
df_parquet.select("estado").distinct().show(32)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|              estado|
+--------------------+
|          tamaulipas|
|           zacatecas|
|          nuevo leon|
|            campeche|
|     san luis potosi|
|            veracruz|
|             morelos|
|          guanajuato|
|              sonora|
|            tlaxcala|
|             nayarit|
|             sinaloa|
|              oaxaca|
|            guerrero|
|        quintana roo|
|           queretaro|
|    estado de mexico|
|              puebla|
|             durango|
|             jalisco|
|      aguascalientes|
|coahuila de zaragoza|
| baja california sur|
|              colima|
|             tabasco|
|           chihuahua|
|     baja california|
|    ciudad de mexico|
|             yucatan|
|             chiapas|
|             hidalgo|
| michoacan de ocampo|
+--------------------+

#### Nº de catálogos por estado y por año. 


In [ ]:
from pyspark.sql import functions as F

#CONSULTA
# Agrupamos directamente por año y estado
df_catalogos_por_estado = df_parquet.groupBy("anio", "estado") \
    .agg(F.countDistinct("catalogo").alias("No_Catalogos")) \
    .orderBy("anio", "estado")

#IMPRESION
rows = df_catalogos_por_estado.collect()

from itertools import groupby

for anio, grupo_anio in groupby(rows, key=lambda r: r['anio']):
    print(f"\n\n{'='*50}")
    print(f"AÑO: {anio}")
    print(f"{'='*50}")
    print(f"{'Estado':<30} {'No. Catalogos':>15}")
    print("-" * 45)
    for row in grupo_anio:
        print(f"{row['estado']:<30} {row['No_Catalogos']:>15}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



A?O: 2018
Estado                           No. Catalogos
---------------------------------------------
aguascalientes                               9
baja california                             10
baja california sur                          9
campeche                                     9
chiapas                                     10
chihuahua                                   10
ciudad de mexico                            10
coahuila de zaragoza                        10
colima                                      10
durango                                     10
estado de mexico                            10
guanajuato                                  10
guerrero                                    10
hidalgo                                     10
jalisco                                     10
michoacan de ocampo                          9
morelos                                      9
nayarit                                     10
nuevo leon                                  10
oa

### Para cada estado: Mayor y Menor número de catálogos por año

Los años mostrados como dónde se alcanzó el máximo y el mínimo no son necesariamente únicos. 

In [ ]:
#De cada estado: obtenemos el menor y el mayor número de catálogos por año. 
#Los años no necesariamente únicos

from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Base: catálogos por estado y año
catalogos_por_estado_anio = df_parquet.groupBy("estado", "anio") \
    .agg(F.countDistinct("catalogo").alias("count"))

# Ventanas por estado ordenadas por count
window_min = Window.partitionBy("estado").orderBy("count")
window_max = Window.partitionBy("estado").orderBy(F.desc("count"))

# Filtramos solo una fila por estado (la de menor y mayor count)
df_min = catalogos_por_estado_anio \
    .withColumn("rn", F.row_number().over(window_min)) \
    .filter(F.col("rn") == 1) \
    .select("estado", F.col("anio").alias("anho_min"), F.col("count").alias("min_catalogos"))

df_max = catalogos_por_estado_anio \
    .withColumn("rn", F.row_number().over(window_max)) \
    .filter(F.col("rn") == 1) \
    .select("estado", F.col("anio").alias("anho_max"), F.col("count").alias("max_catalogos"))

# Unimos resultados
resultado = df_min.join(df_max, on="estado").orderBy("estado")

resultado.show(32, truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+-------------+--------+-------------+
|estado              |anho_min|min_catalogos|anho_max|max_catalogos|
+--------------------+--------+-------------+--------+-------------+
|aguascalientes      |2024    |9            |2019    |11           |
|baja california     |2021    |9            |2022    |11           |
|baja california sur |2018    |9            |2023    |11           |
|campeche            |2024    |9            |2023    |11           |
|chiapas             |2024    |8            |2019    |10           |
|chihuahua           |2020    |5            |2019    |11           |
|ciudad de mexico    |2020    |9            |2023    |11           |
|coahuila de zaragoza|2020    |9            |2022    |11           |
|colima              |2020    |4            |2019    |10           |
|durango             |2024    |9            |2022    |11           |
|estado de mexico    |2020    |9            |2023    |11           |
|guanajuato          |2024    |9  

## Del catálogo BASICOS

Las preguntas que se muestran en esta sección se refieren sólo al catálog **Básicos** que fue el asignado a nuestro grupo. 

Para acelerar las búsquedas, creamos un primer dataframe filtrado por catálogo, para así no tener que especificar siempre el catálogo. 

In [ ]:
#¿Cuańtas marcas diferentes tiene tu categoría?(basicos)
CATALOG_TYPE = "basicos"
df_basicos = df_parquet.filter(col("catalogo") == CATALOG_TYPE)
df_basicos.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- producto: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- precio: float (nullable = true)
 |-- fecha: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- anio: integer (nullable = true)

### Número de marcas diferentes que aparecen en nuestra categoría. 

In [ ]:
#Contamos número de marcas
df_basicos.select("marca").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

637

Las preguntas que se hacen en esta sección hablan de marcas. En nuestro caso, no tiene mucho sentido mostrar sólo la marca, pues se puede referir a productos muy diversos. Así pues, consideramos siempre el par **producto-marca**

### Marca con mayor precio

##### ¿Cuál es la marca con mayor precio? ¿En qué estado se encuentra?
Como tenemos información de varios años, consideraremos el precio promedio.


In [ ]:
from pyspark.sql.functions import avg
# ¿Cuál es la marca con mayor precio? ¿En qué estado?
#Vamos a contestar al producto, pues la marca no tiene mucho sentido. 
mayor_promedio = df_basicos.groupBy("producto", "marca", "estado")\
                            .agg(avg("precio").alias("precio_prom"))\
                            .orderBy(col("precio_prom").desc())
mayor_promedio.show(15)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------+-------------------+-----------------+
|      producto|     marca|             estado|      precio_prom|
+--------------+----------+-------------------+-----------------+
|copa menstrual|diva cup 2|       quintana roo|723.3333333333334|
|copa menstrual|diva cup 2|           veracruz|            690.0|
|copa menstrual|diva cup 2|          queretaro| 677.452380952381|
|copa menstrual|diva cup 2|             puebla|            674.0|
|copa menstrual|diva cup 2|baja california sur|644.0666666666667|
|copa menstrual|diva cup 2|          zacatecas|634.2105263157895|
|copa menstrual|diva cup 2|   ciudad de mexico|628.6392307491093|
|copa menstrual|diva cup 1|             puebla|628.3589743589744|
|copa menstrual|diva cup 1|          chihuahua|619.9000244140625|
|copa menstrual|diva cup 1|          queretaro|615.6611111111112|
|copa menstrual|diva cup 1|            durango|615.5914515904018|
|copa menstrual|diva cup 1|          zacatecas|614.4038614126353|
|copa mens

Por los resultados observados el producto más caro es la **Copa Menstrual** en el estado de **Quintana Roo**, además observamos que este producto está en el top 10 de los más caros, ya que la tabla anterior está ordenada por precio promedio, con lo que si hubiera en el estado de Veracruz un producto más barato que la copa menstrual en el estado de Querétaro, aparcería en tercer lugar. 

### En la CDMX
Queremos saber cuáles son los productos más baratos. De nuevo tomamos el promedio anual. 

In [ ]:
from pyspark.sql.functions import min
# ¿Cuál es el producto/marca con menor precio promedio en CDMX?
menor_precio_promedio_cdmx = df_basicos.filter(col("estado") == "ciudad de mexico")\
    .groupBy("producto", "marca")\
    .agg(avg("precio").alias("precio_promedio"))\
    .orderBy("precio_promedio")

menor_precio_promedio_cdmx.show(5, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+--------+------------------+
|producto                |marca   |precio_promedio   |
+------------------------+--------+------------------+
|pan blanco bolillo      |s/m     |1.7916649643842255|
|polvo p/preparar bebidas|frutimax|2.3564714899777206|
|polvo p/preparar bebidas|clight  |3.165304193614619 |
|pasta para sopa         |soriana |3.3124000021616617|
|polvo p/preparar bebidas|livean  |3.331467661988083 |
+------------------------+--------+------------------+
only showing top 5 rows

## Marcas con más observaciones

Hacemos ahora un análisis de las marcas que tienen más observaciones, a nivel nacional 


In [ ]:
# ¿Cuál es la marca con mayores observaciones?
df_basicos.groupBy("marca")\
        .agg(F.count("*").alias("observaciones"))\
        .orderBy(col("observaciones").desc())\
        .show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------+
|marca|observaciones|
+-----+-------------+
|  s/m|      2762039|
+-----+-------------+
only showing top 1 row

## Análisis de los precios por marca. 

De nuevo consideramos el precio promedio a lo largo de los años por producto-marca, pues nos parece más explicativo. 

#### ¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?

In [ ]:
#¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from itertools import groupby


#CONSULTA
# 1. Promedio de precio por estado, producto y marca
df_avg = df_basicos.groupBy("estado", "producto", "marca") \
    .agg(F.avg("precio").alias("precio_promedio"))

# 2. Crear una ventana por estado, ordenando por precio promedio descendente
window = Window.partitionBy("estado").orderBy(F.desc("precio_promedio"))

# 3. Agregar ranking
df_ranked_avg = df_avg.withColumn("rank", F.row_number().over(window))

# 4. Filtrar top 5 por estado
df_top5_avg = df_ranked_avg.filter(col("rank") <= 5).orderBy("estado")

#IMPRESION

rows = df_top5_avg.select("estado", "producto", "marca", "precio_promedio", "rank").collect()

for estado, grupo_estado in groupby(rows, key=lambda r: r['estado']):
    print(f"\n\n{'='*50}")
    print(f"ESTADO: {estado.upper()}")
    print(f"{'='*50}")
    print(f"{'Producto':<30} {'Marca':<25} {'Precio Promedio':>18}")
    print("-" * 75)
    for row in grupo_estado:
        print(f"{row['producto']:<30} {row['marca']:<25} {row['precio_promedio']:>18.2f}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



ESTADO: AGUASCALIENTES
Producto                       Marca                        Precio Promedio
---------------------------------------------------------------------------
copa menstrual                 saba cup                              573.35
copa menstrual                 diva cup 2                            541.11
copa menstrual                 diva cup 1                            505.58
nuez                           s/m                                   422.31
tequila                        jose cuervo tradicional               368.82


ESTADO: BAJA CALIFORNIA
Producto                       Marca                        Precio Promedio
---------------------------------------------------------------------------
copa menstrual                 diva cup 1                            550.38
copa menstrual                 saba cup                              543.08
nuez                           s/m                                   473.05
almendras                      s/m   

Con esta visualización es más fácil darse cuenta que el producto básico más caro es la copa menstrual, y en estados como Campeche, donde sólo parece que venden una marca de copa menstrual, entonces aparecen otros productos como la nuez, el tequila o el camarón. 

#### ¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)


In [ ]:
df_cdmx = df_basicos.filter(col("estado") == "ciudad de mexico")\
            .groupBy("producto", "marca") \
            .agg(F.avg("precio").alias("precio_promedio"))\
            .orderBy("precio_promedio")\
            .show(5, truncate = False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+--------+------------------+
|producto                |marca   |precio_promedio   |
+------------------------+--------+------------------+
|pan blanco bolillo      |s/m     |1.7916649643842255|
|polvo p/preparar bebidas|frutimax|2.3564714899777206|
|polvo p/preparar bebidas|clight  |3.165304193614619 |
|pasta para sopa         |soriana |3.3124000021616617|
|polvo p/preparar bebidas|livean  |3.331467661988083 |
+------------------------+--------+------------------+
only showing top 5 rows

####  ¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?


In [ ]:
# ¿Cuál es la marca con mayores observaciones?
df_basicos.groupBy("marca")\
        .agg(F.count("*").alias("observaciones"))\
        .orderBy(col("observaciones").desc())\
        .show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+
|     marca|observaciones|
+----------+-------------+
|       s/m|      2762039|
|la costena|      1311439|
|       fud|      1151109|
|     bimbo|       819491|
|      lala|       750135|
+----------+-------------+
only showing top 5 rows

Veamos ahora las observaciones por estado. 

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from itertools import groupby

#CONSULTA
# Agrupamos por estado y marca, y contamos observaciones
df_marca_estado = df_basicos.groupBy("estado", "marca") \
    .agg(F.count("*").alias("observaciones"))

# Definimos una ventana particionada por estado y ordenada por observaciones desc
window_estado = Window.partitionBy("estado").orderBy(F.desc("observaciones"))

# Aplicamos una fila numérica para rankear
df_top5 = df_marca_estado.withColumn("rank", F.row_number().over(window_estado)) \
                         .filter(F.col("rank") <= 5) \
                         .orderBy("estado", "rank")

#IMPRESION

rows = df_top5.select("estado", "marca", "observaciones", "rank").collect()

for estado, grupo_estado in groupby(rows, key=lambda r: r['estado']):
    print(f"\n\n{'='*50}")
    print(f"ESTADO: {estado.upper()}")
    print(f"{'='*50}")
    print(f"{'Marca':<25} {'Observaciones':>10}")
    print("-" * 50)
    for row in grupo_estado:
        print(f"{row['marca']:<25} {row['observaciones']:>10.2f}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



ESTADO: AGUASCALIENTES
Marca                     Observaciones
--------------------------------------------------
s/m                         52322.00
fud                         20041.00
la costena                  18908.00
bimbo                       15595.00
lala                        12561.00


ESTADO: BAJA CALIFORNIA
Marca                     Observaciones
--------------------------------------------------
s/m                         47708.00
la costena                  21335.00
fud                         15562.00
bimbo                       14915.00
lala                        13911.00


ESTADO: BAJA CALIFORNIA SUR
Marca                     Observaciones
--------------------------------------------------
s/m                         44265.00
la costena                  20512.00
fud                         17289.00
bimbo                       13981.00
lala                        13378.00


ESTADO: CAMPECHE
Marca                     Observaciones
--------------------------------

De la primera salida, observamos más claramente que en la mayoría de los casos, las marcas más observadas son **"s/m"="Sin Marca"**, **fud** y **la costeña**. En algunos estados cambia el orden del 2º y 3º, y en otros sólo aparece uno de los dos en esas posiciones, ocupando la faltante otras marcas. 

##### ¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?

In [ ]:
# Paso 1: obtener el máximo año registrado
max_anio = df_basicos.agg(F.max("anio")).first()[0]

# Paso 2: obtener presencia de cada marca por año
df_marca_anio = df_basicos.select("marca", "anio").distinct()

# Paso 3: obtener el último año en que se vio cada marca
df_ultimo_anio = df_marca_anio.groupBy("marca") \
    .agg(F.max("anio").alias("ultimo_anio"))

# Paso 4: detectar marcas que dejaron de aparecer
df_marcas_desaparecidas = df_ultimo_anio.filter(F.col("ultimo_anio") < max_anio) \
                                        .orderBy("ultimo_anio", "marca")

df_marcas_desaparecidas.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------+-----------+
|marca                                        |ultimo_anio|
+---------------------------------------------+-----------+
|alpino. pavino.                              |2018       |
|big cola                                     |2018       |
|chalet                                       |2018       |
|d calidad chedraui                           |2018       |
|kleen bebe. comodisec max                    |2018       |
|lala. siluette plus 0%                       |2018       |
|manzana lift                                 |2018       |
|morelos. independiente                       |2018       |
|morelos. independiente o mexicano tradicional|2018       |
|morelos. insurgente                          |2018       |
|nochebuena                                   |2018       |
|osram. dulux el micro twist. 10 anos         |2018       |
|philips. mini twister. t2. 8 anos            |2018       |
|rosa blanca                            